# Activation patching basic


In [1]:

%load_ext autoreload
%autoreload 2

device = "cuda:3"
starcoderbase_1b = "/home/arjun/models/starcoderbase-1b/"

from nnsight import LanguageModel
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from src.utils import *
import datasets
from src.experiments.wrong_type_patching import *
from src.experiments.type_patching import *
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
import plotly.express as px
from src.patching import *
from nnsight import LanguageModel, util
from nnsight.tracing.Proxy import Proxy
import pandas as pd
import datasets
from src.utils import *
import regex as re
import circuitsvis as cv

In [2]:
model = LanguageModel(starcoderbase_1b, device_map=device)
model, model.config

(GPTBigCodeForCausalLM(
   (transformer): GPTBigCodeModel(
     (wte): Embedding(49152, 2048)
     (wpe): Embedding(8192, 2048)
     (drop): Dropout(p=0.1, inplace=False)
     (h): ModuleList(
       (0-23): 24 x GPTBigCodeBlock(
         (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
         (attn): GPTBigCodeSdpaAttention(
           (c_attn): Linear(in_features=2048, out_features=2304, bias=True)
           (c_proj): Linear(in_features=2048, out_features=2048, bias=True)
           (attn_dropout): Dropout(p=0.1, inplace=False)
           (resid_dropout): Dropout(p=0.1, inplace=False)
         )
         (ln_2): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
         (mlp): GPTBigCodeMLP(
           (c_fc): Linear(in_features=2048, out_features=8192, bias=True)
           (c_proj): Linear(in_features=8192, out_features=2048, bias=True)
           (act): PytorchGELUTanh()
           (dropout): Dropout(p=0.1, inplace=False)
         )
       )
     )
     (ln_f

## Investigating what happens when a model gives the wrong prediction

Here we give an approximate algorithm of what the model may be doing:

1. identify first untyped variable and bind it to a value of the same type by transfering info from value to variable
2. repeat 1 until all variables are typed
3. copy the answer onto `<fim_middle>`

In [3]:
ds = datasets.load_dataset("franlucc/starcoderbase-1b-wrong-type-patching-v0-seed42", split="train")
working_ds = ds.filter(lambda x: x["success"])
broken_ds = ds.filter(lambda x: not x["success"])
ds, working_ds, broken_ds

(Dataset({
     features: ['prompt', 'solution', 'generated', 'success', 'filename', 'variation_idx'],
     num_rows: 300
 }),
 Dataset({
     features: ['prompt', 'solution', 'generated', 'success', 'filename', 'variation_idx'],
     num_rows: 163
 }),
 Dataset({
     features: ['prompt', 'solution', 'generated', 'success', 'filename', 'variation_idx'],
     num_rows: 137
 }))

## looking at one failing example

In [4]:
from IPython.display import Code
content = broken_ds[0]["prompt"]
solution = broken_ds[0]["solution"]
generated = broken_ds[0]["generated"]
print(solution, "\n\n", generated)
content = content.split("function _uniq_7")[0]

rename = {
    "_uniq_0": "_uniq_q",
    "_uniq_1[^\d]": "_uniq_t",
}
def rename_vars(content, rename):
    for old, new in rename.items():
        content = re.sub(old, new, content)
    return content

content = rename_vars(content, rename)
solution = rename_vars(solution, rename)
generated = rename_vars(generated, rename)
print("SOLUTION:", solution, "\n\n", "GENERATED:", generated)

code = Code(content, language="typescript")
base_prompt = placeholder_to_std_fmt(content, STARCODER_FIM)
code

_uniq_0 

 _uniq_1) => _uniq_0,
  _uniq_11: _uniq_0
): _uniq_0 {
  switch (_uniq_9.kind) {
    case "Ctor4": {
      return _uniq
SOLUTION: _uniq_q 

 GENERATED: _uniq_t => _uniq_q,
  _uniq_11: _uniq_q
): _uniq_q {
  switch (_uniq_9.kind) {
    case "Ctor4": {
      return _uniq


declare var require: any;
const assert = require("node:assert");
type _uniq_q = { kind: "Ctor2" } | { kind: "Ctor3"; f0: _uniq_q };
type _uniq_t= { kind: "Ctor4" } | { kind: "Ctor5"; f0: _uniq_q; f1: _uniq_t};

function _uniq_6(
  _uniq_9: _uniq_t
  _uniq_10: (__x7: _uniq_q, __x8: <FILL>) => _uniq_q,
  _uniq_11: _uniq_q
): _uniq_q {
  switch (_uniq_9.kind) {
    case "Ctor4": {
      return _uniq_11;
    }
    case "Ctor5": {
      let _uniq_13 = _uniq_9.f1;
      let _uniq_12 = _uniq_9.f0;
      return _uniq_6(_uniq_13, _uniq_10, _uniq_10(_uniq_11, _uniq_12));
    }
  }
}

In [6]:
prompt = "function" +base_prompt.split("function")[-1]

max_new_tokens = 4

def get_final_states(prompt, max_new_tokens):
    with model.generate(max_new_tokens=max_new_tokens) as generator:
        with generator.invoke(input=prompt) as invoker:
            hidden_states = []
            for i in range(max_new_tokens):
                final_hs = model.lm_head.output
                print(model.transformer.h[5].mlp.output.shape)
                hidden_states.append(final_hs.save())
                invoker.next()
    return hidden_states

hidden_states = get_final_states(prompt, max_new_tokens)

torch.Size([1, 172, 2048])
torch.Size([1, 1, 2048])
torch.Size([1, 1, 2048])
torch.Size([1, 1, 2048])


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [60]:
out = [util.apply(i, lambda x: x.value, Proxy) for i in hidden_states]

topk=10
predictions = []
for j,hs in enumerate(out):
    if j == 0:
        hs = hs[:, -1, :]
        hs = hs.unsqueeze(1)
    top_vals_and_idx = hs.softmax(dim=-1).topk(topk)
    indices = top_vals_and_idx.indices[0][0].tolist()
    values = top_vals_and_idx.values.tolist()[0][0]
    top_idx = [model.tokenizer.decode(i) for i in indices]
    predictions.append(list(zip(top_idx, values)))

greedy_prediction = "".join([p[0][0] for p in predictions])
print(greedy_prediction)
result_table = pd.DataFrame(predictions, columns=[f"top_pred_{i}" for i in range(topk)], index=[f"next_token_{i}" for i in range(max_new_tokens)])
result_table

_uniq_q


,top_pred_0,top_pred_1,top_pred_2,top_pred_3,top_pred_4,top_pred_5,top_pred_6,top_pred_7,top_pred_8,top_pred_9
next_token_0,"(_, 0.8836920261383057)","(__, 0.04612018167972565)","(string, 0.01082543283700943)","((, 0.009049000218510628)","((_, 0.006661245133727789)","(number, 0.005222067702561617)","(any, 0.002787975361570716)","((__, 0.0024021132849156857)","(<|endoftext|>, 0.0022207931615412235)","("", 0.0017489035381004214)"
next_token_1,"(uniq, 0.9975231289863586)","(q, 0.0001862113713286817)","(t, 9.274380136048421e-05)","(a, 9.199933265335858e-05)","(uni, 9.078498260350898e-05)","(distinct, 8.081931446213275e-05)","(unique, 7.646429730812088e-05)","(u, 4.0061382605927065e-05)","(x, 3.667894270620309e-05)","(Q, 3.614146771724336e-05)"
next_token_2,"(_, 0.9995818734169006)","(__, 0.0001161865220637992)","(2, 3.273446054663509e-05)","(<|endoftext|>, 2.614452205307316e-05)","(q, 1.9497674657031894e-05)","(), 1.9011838958249427e-05)","(_), 1.5274810721166432e-05)","(1, 1.4974726582295261e-05)","(,, 1.312915173912188e-05)","(., 1.0743523489509244e-05)"
next_token_3,"(q, 0.6972823143005371)","(t, 0.1781110316514969)","(p, 0.02561504766345024)","(r, 0.014788011088967323)","(s, 0.00788042787462473)","(1, 0.007198556326329708)","(a, 0.006413357798010111)","(w, 0.003968821372836828)","(u, 0.0035089619923382998)","(b, 0.003295798785984516)"


In [7]:
## Sanity check generation

# from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
# tokenizer = AutoTokenizer.from_pretrained(starcoderbase_1b)
# llm = AutoModelForCausalLM.from_pretrained(starcoderbase_1b).to(device)

In [8]:
# max_new_tokens = 4
# tokens = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
# out = llm.generate(tokens, max_new_tokens=max_new_tokens, do_sample=False)
# text = [tokenizer.decode(i) for i in out.tolist()]
# text = "".join(text)
# text.split("<fim_middle>")[-1]

In [9]:
saved_tokens = []
def get_all_hidden_states(prompt, max_new_tokens):
    with model.generate(max_new_tokens=max_new_tokens) as generator:
        with generator.invoke(input=prompt) as invoker:
            saved_tokens.append(invoker.input.tokens())
            print(saved_tokens)
            hidden_states = {layer : [] for layer in range(model.config.n_layer)}
            for i in range(max_new_tokens-1):
                invoker.next()
                
            for l in range(model.config.n_layer):
                hs = model.transformer.h[l].output[0].save()
                hidden_states[l].append(model.lm_head(model.transformer.ln_f(hs)).save())
    return hidden_states

full_hidden_states = get_all_hidden_states(prompt, max_new_tokens)
value_hidden_states = {k : [util.apply(i, lambda x: x.value, Proxy) for i in v] for k,v in full_hidden_states.items()}


[['<fim_prefix>', 'declare', 'Ġvar', 'Ġrequire', ':', 'Ġany', ';', 'Ċ', 'const', 'Ġassert', 'Ġ=', 'Ġrequire', '("', 'node', ':', 'assert', '");', 'Ċ', 'type', 'Ġ_', 'uniq', '_', 'q', 'Ġ=', 'Ġ{', 'Ġkind', ':', 'Ġ"', 'Ctor', '2', '"', 'Ġ}', 'Ġ|', 'Ġ{', 'Ġkind', ':', 'Ġ"', 'Ctor', '3', '";', 'Ġf', '0', ':', 'Ġ_', 'uniq', '_', 'q', 'Ġ};', 'Ċ', 'type', 'Ġ_', 'uniq', '_', 't', '=', 'Ġ{', 'Ġkind', ':', 'Ġ"', 'Ctor', '4', '"', 'Ġ}', 'Ġ|', 'Ġ{', 'Ġkind', ':', 'Ġ"', 'Ctor', '5', '";', 'Ġf', '0', ':', 'Ġ_', 'uniq', '_', 'q', ';', 'Ġf', '1', ':', 'Ġ_', 'uniq', '_', 't', '};', 'Ċ', 'Ċ', 'function', 'Ġ_', 'uniq', '_', '6', '(', 'ĊĠ', 'Ġ_', 'uniq', '_', '9', ':', 'Ġ_', 'uniq', '_', 't', 'ĊĠ', 'Ġ_', 'uniq', '_', '1', '0', ':', 'Ġ(__', 'x', '7', ':', 'Ġ_', 'uniq', '_', 'q', ',', 'Ġ__', 'x', '8', ':', 'Ġ', '<fim_suffix>', ')', 'Ġ=>', 'Ġ_', 'uniq', '_', 'q', ',', 'ĊĠ', 'Ġ_', 'uniq', '_', '1', '1', ':', 'Ġ_', 'uniq', '_', 'q', 'Ċ', '):', 'Ġ_', 'uniq', '_', 'q', 'Ġ{', 'ĊĠ', 'Ġswitch', 'Ġ(_', 'uniq', '_', '

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [10]:
tok_preds = {layer : [] for layer in range(model.config.n_layer)}
# value_hidden_states[14]
for layer, hs in value_hidden_states.items():
    tok_hs = hs[0]
    top_vals_and_idx = tok_hs.softmax(dim=-1).topk(topk)
    indices = top_vals_and_idx.indices.tolist()[0][0]
    values = top_vals_and_idx.values.tolist()[0][0]
    top_idx = [model.tokenizer.decode(i) for i in indices]
    tok_preds[layer].append(list(zip(top_idx, values)))
    
tok_preds_df = pd.DataFrame(tok_preds)
tok_preds_df.columns = [f"layer_{i}" for i in range(model.config.n_layer)]
tok_preds_df

,layer_0,layer_1,layer_2,layer_3,layer_4,layer_5,layer_6,layer_7,layer_8,layer_9,...,layer_14,layer_15,layer_16,layer_17,layer_18,layer_19,layer_20,layer_21,layer_22,layer_23
0,"[(ativos, 0.023885412141680717), (áveis, 0.014...","[(eded, 0.04464365169405937), (cheme, 0.040472...","[(gular, 0.18594110012054443), (iteral, 0.0508...","[(gular, 0.26038503646850586), (ayload, 0.0385...","[(ayload, 0.04933283478021622), (gular, 0.0417...","[(ayload, 0.11017901450395584), (chemy, 0.0570...","[(ayload, 0.11401428282260895), (hibited, 0.05...","[(opx, 0.07060158252716064), (ilon, 0.05417048...","[(edata, 0.02658381685614586), (ookie, 0.01994...","[(nown, 0.059704143553972244), (edata, 0.05822...",...,"[(<|endoftext|>, 0.42255502939224243), (utoff,...","[(<|endoftext|>, 0.9999828338623047), (<fim_pa...","[(<|endoftext|>, 0.9999886751174927), (<fim_pa...","[(<|endoftext|>, 0.999969482421875), (t, 1.307...","[(t, 0.7486408352851868), (<|endoftext|>, 0.10...","[(t, 0.7072942852973938), (q, 0.28672108054161...","[(q, 0.7877132296562195), (t, 0.21189612150192...","[(t, 0.6726053953170776), (q, 0.32711467146873...","[(q, 0.5214332342147827), (t, 0.47795587778091...","[(t, 0.5981452465057373), (q, 0.39379426836967..."


In [11]:
## look at logits in prompt and how they change through layers
# for each layer, collect the topk predictions for each token in the prompt

prompt_tok_preds = {l : [] for l in range(model.config.n_layer)}
for layer, hs in value_hidden_states.items():
    prompt_hs = hs[0]
    for k in range(prompt_hs.shape[1]):
        tok_hs = prompt_hs[:, k, :]
        top_vals_and_idx = tok_hs.softmax(dim=-1).topk(topk)
        indices = top_vals_and_idx.indices[0].tolist()
        values = top_vals_and_idx.values.tolist()[0]
        top_idx = [model.tokenizer.decode(i) for i in indices]
        prompt_tok_preds[layer].append(list(zip(top_idx, values)))

In [12]:
prompt_logits = pd.DataFrame(prompt_tok_preds)
prompt_logits.columns = [f"layer_{i}" for i in range(model.config.n_layer)]
prompt_logits

,layer_0,layer_1,layer_2,layer_3,layer_4,layer_5,layer_6,layer_7,layer_8,layer_9,...,layer_14,layer_15,layer_16,layer_17,layer_18,layer_19,layer_20,layer_21,layer_22,layer_23
0,"[(ativos, 0.023885412141680717), (áveis, 0.014...","[(eded, 0.04464365169405937), (cheme, 0.040472...","[(gular, 0.18594110012054443), (iteral, 0.0508...","[(gular, 0.26038503646850586), (ayload, 0.0385...","[(ayload, 0.04933283478021622), (gular, 0.0417...","[(ayload, 0.11017901450395584), (chemy, 0.0570...","[(ayload, 0.11401428282260895), (hibited, 0.05...","[(opx, 0.07060158252716064), (ilon, 0.05417048...","[(edata, 0.02658381685614586), (ookie, 0.01994...","[(nown, 0.059704143553972244), (edata, 0.05822...",...,"[(<|endoftext|>, 0.42255502939224243), (utoff,...","[(<|endoftext|>, 0.9999828338623047), (<fim_pa...","[(<|endoftext|>, 0.9999886751174927), (<fim_pa...","[(<|endoftext|>, 0.999969482421875), (t, 1.307...","[(t, 0.7486408352851868), (<|endoftext|>, 0.10...","[(t, 0.7072942852973938), (q, 0.28672108054161...","[(q, 0.7877132296562195), (t, 0.21189612150192...","[(t, 0.6726053953170776), (q, 0.32711467146873...","[(q, 0.5214332342147827), (t, 0.47795587778091...","[(t, 0.5981452465057373), (q, 0.39379426836967..."


In [13]:
# curr_layer = prompt_logits["layer_23"].tolist()
# print(len(curr_layer))

# probabilities = []
# text = []
# x = []
# minn = 251
# for k in range(minn, minn+10):
#     token = curr_layer[k]
#     token_logit_text = [i[0] for i in token]
#     token_logit_prob = [i[1] for i in token]
#     probabilities.append(token_logit_prob)
#     x.append(saved_tokens[0][k])
#     text.append(token_logit_text)
    
# import plotly.graph_objects as go

# fig = go.Figure(data=go.Heatmap(
#                     z=probabilities,
#                     x = x,
#                     text=text,
#                     texttemplate="%{text}",
#                     textfont={"size":15}))


# fig.show()

Reasons why prediction is wrong
- 0 is similar to 1 (number issue)
    - probably not, can change 0 to q and 1 to t and it still fails
- another algorithm picks 1

In [14]:

print(f"====Generated:====\n\n{generated}\n\n====Solution:=====\n\n{solution}")
code

====Generated:====

_uniq_t => _uniq_q,
  _uniq_11: _uniq_q
): _uniq_q {
  switch (_uniq_9.kind) {
    case "Ctor4": {
      return _uniq

====Solution:=====

_uniq_q


declare var require: any;
const assert = require("node:assert");
type _uniq_q = { kind: "Ctor2" } | { kind: "Ctor3"; f0: _uniq_q };
type _uniq_t= { kind: "Ctor4" } | { kind: "Ctor5"; f0: _uniq_q; f1: _uniq_t};

function _uniq_6(
  _uniq_9: _uniq_t
  _uniq_10: (__x7: _uniq_q, __x8: <FILL>) => _uniq_q,
  _uniq_11: _uniq_q
): _uniq_q {
  switch (_uniq_9.kind) {
    case "Ctor4": {
      return _uniq_11;
    }
    case "Ctor5": {
      let _uniq_13 = _uniq_9.f1;
      let _uniq_12 = _uniq_9.f0;
      return _uniq_6(_uniq_13, _uniq_10, _uniq_10(_uniq_11, _uniq_12));
    }
  }
}

# Attn visualization

In [15]:
# TODO something wrong here, always gives same resd
attn, toks = attention_vis(model, prompt+"_uniq_")
print(len(attn), torch.stack(attn).sum().item())
# b, h, q, k

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
GPTBigCodeModel is using GPTBigCodeSdpaAttention, but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True` and `head_mask` not None. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


24 101376.0


In [16]:
from sklearn.preprocessing import normalize


def normalize_attn(attention_matrix):
    return normalize(attention_matrix, norm="l2", axis=1)

print(attn[0].shape)
normalize_attn(attn[13][3].cpu().detach().numpy()), attn[13][3]

torch.Size([16, 264, 264])


(array([[1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00],
        [9.9998385e-01, 5.6785061e-03, 0.0000000e+00, ..., 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00],
        [9.9999410e-01, 1.9448635e-03, 2.8284541e-03, ..., 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00],
        ...,
        [8.2785435e-02, 1.6548799e-04, 9.0345355e-05, ..., 2.8032113e-03,
         0.0000000e+00, 0.0000000e+00],
        [8.4791221e-02, 6.9219786e-05, 3.4148121e-05, ..., 2.1113926e-03,
         2.6780353e-03, 0.0000000e+00],
        [1.2193324e-01, 6.7537971e-05, 3.2085631e-05, ..., 7.3519088e-03,
         1.7220268e-03, 5.8213212e-03]], dtype=float32),
 tensor([[1.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [9.9435e-01, 5.6465e-03, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [9.9525e-01, 1.9356e-03, 2.8150e-03,  ..., 0.0000e+00, 0.0000e+00,
          0.

In [17]:
# cv.attention.attention_pattern(tokens=no_special_tok[window_idx:], attention=attn[14][window_idx:], mask_upper_tri=True)
# query is destination, key is source

# heatmap
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(15,15))

def create_attention_heatmap(attention_matrix, tokens, row_idx, column_idx, step=10):
    normalized_attention = normalize_attn(attention_matrix)
    
    # Get a subset of the attention matrix
    subset_matrix = normalized_attention[row_idx:row_idx+step, column_idx:column_idx+step]
    xtokens = tokens[column_idx:column_idx+step]
    ytokens = tokens[row_idx:row_idx+step]

    
    # Create a heatmap using Seaborn
    sns.set()

    sns.heatmap(subset_matrix, cmap="viridis", xticklabels=xtokens, yticklabels=ytokens)
    # annot=subset_matrix, fmt=".2f", annot_kws={"size": 10}
    
    plt.xlabel("Quert DST Tokens")
    plt.ylabel("Key SRC Tokens")



layers = []
heads = list(range(16))
window_range = list(range(0, len(toks)-10, 10))

column_idx = len(toks)-10
step = 10
full_sum = 0

os.makedirs("images", exist_ok=True)
    
for layer in tqdm(layers):
    os.makedirs(f"images/layer_{layer}", exist_ok=True)

    for row_idx in window_range:
        xtokens = {i:t for i,t in enumerate(toks[column_idx:column_idx+step])}
        ytokens = {i:t for i,t in enumerate(toks[row_idx:row_idx+step])}
        # print("X DST:\n",xtokens, "\nY SRC:\n", ytokens)
        for h in heads:
            # create a subplot for this head    
            plt.subplot(4,4, h+1)
            kq = attn[layer][h].cpu().detach().numpy()
            
            summ = kq[row_idx:row_idx+step, column_idx:column_idx+step].sum().item()
            if summ == 0:
                continue
            full_sum += summ
            create_attention_heatmap(kq, toks, row_idx=row_idx, column_idx=column_idx, step=step)
        
        if full_sum > 0:
            # print(f"Saving {h} head at row {row_idx}")
            full_sum = 0
            # save
            plt.tight_layout()
            with open(f"images/layer_{layer}/row_{row_idx}_column_{column_idx}_step_{step}.txt", "w") as f:
                f.write(f"X DST:\n{xtokens}\nY SRC:\n{ytokens}")
            plt.savefig(f"images/layer_{layer}/row_{row_idx}_column_{column_idx}_step_{step}.png")
        


0it [00:00, ?it/s]


<Figure size 1500x1500 with 0 Axes>

## Layer erase - logit difference

erasing doesn't quite work -> maybe try noise or random patch?

In [57]:
def get_final_states_erase(prompt, max_new_tokens, layers = []):
    with model.generate(max_new_tokens=max_new_tokens) as generator:
        with generator.invoke(input=prompt) as invoker:
            hidden_states = []
                    
            for i in range(max_new_tokens):

                for l in range(model.config.n_layer):
                    if l in layers:
                        if l == 0:
                            raise ValueError("Cannot erase layer 0")
                        else:
                            model.transformer.h[l].output = last_saved
                    else:
                        last_saved = model.transformer.h[l].output.save()
                final_hs = model.lm_head.output
                hidden_states.append(final_hs.save())
                invoker.next()
             
            hidden_states.append(final_hs.save())
            
    return hidden_states


hidden_states = get_final_states(prompt, 4)
hidden_states_erase = get_final_states_erase(prompt, 4, layers=list(range(2,24)))

out = [util.apply(i, lambda x: x.value, Proxy) for i in hidden_states]
out_erase = [util.apply(i, lambda x: x.value, Proxy) for i in hidden_states_erase]

logit_diffs_q = []
logit_diffs_t = []

q_idx = model.tokenizer.encode("q")[0]
t_idx = model.tokenizer.encode("t")[0]

topk=10
predictions = []
for j in range(len(out)):
    hs = out[j]
    hs_erase = out_erase[j]
    if j == 0:
        hs = hs[:, -1,:]
        hs = hs.unsqueeze(1)
        hs_erase = hs_erase[:, -1,:]
        hs_erase = hs_erase.unsqueeze(1)
        
    logits_clean = hs.softmax(dim=-1)
    logits_erase = hs_erase.softmax(dim=-1)
    
    logit_clean_q = logits_clean[:,:,q_idx].item()
    logit_erase_q = logits_erase[:,:,q_idx].item()
    logit_clean_t = logits_clean[:,:,t_idx].item()
    logit_erase_t = logits_erase[:,:,t_idx].item()
    
    max_logit_clean = logits_clean.argmax().item()
    max_logit_erase = logits_erase.argmax().item()
    
    logit_diffs_q.append([logit_clean_q - logit_erase_q, logit_clean_q, logit_erase_q])
    logit_diffs_t.append([logit_clean_t - logit_erase_t, logit_clean_t, logit_erase_t])
    
erase_prediction = model.tokenizer.decode(max_logit_erase)
clean_prediction = model.tokenizer.decode(max_logit_clean)
print(f"Clean: {clean_prediction}, Erase: {erase_prediction}")

result_table = pd.DataFrame([logit_diffs_q[-1], logit_diffs_t[-1]], index=["q","t"], columns=["diff", "clean", "erase"])
result_table

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


ValueError: Accessing Proxy value before it's been set.